# Playground

In [587]:
import pandas as pd
import pycountry
import dash
import os
from dash.dependencies import Input, Output
import plotly.express as px
from dash import dcc, html, Input, Output, State
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import sys


In [588]:
# load feather file from input directory
dirname = sys.path[0]
df = pd.read_feather(os.path.join(dirname, "../output/df_allyears"))

In [589]:
def alpha2_to_alpha3(alpha_2):
    """Converts ISO 2 country code to ISO 3 country code"""
    country = pycountry.countries.get(alpha_2=alpha_2)
    return country.alpha_3 if country else alpha_2


In [590]:
## Adds country name to dataframe
country_mapping = {country.alpha_3: country.name for country in pycountry.countries}


In [591]:
df["Country"] = df["Country"].apply(alpha2_to_alpha3)


In [592]:
df["Country_Name"] = df["Country"].map(country_mapping)


In [593]:
not_converted = df[df["Country"].str.len() == 2]["Country"].unique()

# Drop rows where 'Country' column contains 2-letter codes
df = df[df["Country"].str.len() != 2]


In [594]:
# rename column earlist_year to year
df.rename(columns={"earliest_filing_year": "Year"}, inplace=True)


In [595]:
# Retrieves alle ISO 3 country codes and country names from pycountry
countries_df = pd.DataFrame(
    {
        "Country": [country.alpha_3 for country in pycountry.countries],
        "Country_Name": [country.name for country in pycountry.countries],
    }
)


In [596]:
# create a DataFrame that includes all years in your original dataset
years_df = pd.DataFrame({"Year": df["Year"].unique()})


In [597]:
# create a DataFrame that includes every combination of country and year.
countries_years_df = pd.merge(
    countries_df.assign(key=0), years_df.assign(key=0), on="key"
).drop("key", axis=1)


In [598]:
# merge this countries_years_df DataFrame with original df
df = pd.merge(
    countries_years_df, df, on=["Country", "Country_Name", "Year"], how="outer"
)


In [599]:
# Replace NaN values with 0
df["ff"] = df["ff"].fillna(0)
df["renewables"] = df["renewables"].fillna(0)
df["share_renewables"] = df["share_renewables"].fillna(0)
df["share_ff"] = df["share_ff"].fillna(0)


In [600]:
# Keep only data from the year 2000 to 2017
df = df[df["Year"] >= 2000]
df = df[df["Year"] <= 2017]


In [601]:
# Transform share in %
df["share_ff"] = df["share_ff"] * 100
df["share_renewables"] = df["share_renewables"] * 100

# Round share_ff and share_renewables to 2 decimals
df["share_ff"] = df["share_ff"].round(2)
df["share_renewables"] = df["share_renewables"].round(2)


In [602]:
# Custom color scale
color_scale_renewables = [
    (0.0, "rgb(211,211,211)"),  # Light grey for 0
    (0.00001, "rgb(229,245,224)"),  # Start shades of green for any value higher than 0
    (0.1, "rgb(199,233,192)"),
    (0.2, "rgb(161,217,155)"),
    (0.3, "rgb(116,196,118)"),
    (0.4, "rgb(65,171,93)"),
    (0.5, "rgb(35,139,69)"),
    (0.6, "rgb(0,109,44)"),
    (0.7, "rgb(0,68,27)"),
    (0.8, "rgb(0,68,27)"),
    (0.9, "rgb(0,68,27)"),
    (1.0, "rgb(0,68,27)"),  # Dark green for all values above 150
]

color_scale_ff = [
    (0.0, "rgb(211,211,211)"),  # Light grey for 0
    (0.00001, "rgb(246,232,195)"),  # Very light beige
    (0.1, "rgb(223,194,125)"),
    (0.2, "rgb(191,129,45)"),
    (0.3, "rgb(141,109,49)"),
    (0.4, "rgb(140,81,10)"),
    (0.5, "rgb(129,89,23)"),
    (0.6, "rgb(103,65,15)"),
    (0.7, "rgb(80,50,20)"),
    (0.8, "rgb(60,38,22)"),
    (0.9, "rgb(40,25,15)"),
    (1.0, "rgb(20,10,5)"),  # Dark brown for all values above 150
]


In [603]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# App Layout
app.layout = html.Div(
    [
        html.Header(
            [
                html.H1("Green Energy Innovation"),
                html.H5(
                    "Work by Ana Brás Monteiro, Theodor Friederiszick, Carole Marullaz"
                ),
            ]
        ),
        html.Main(
            [
                html.P(
                    [
                        "Welcome to our page! ",
                        "Here, you will find a range of interactive world maps showcasing the progress of energy innovation.",
                    ],
                    className="main-text",
                ),
                html.P(
                    [
                        "Our maps show the number of patents filed globally for both renewable and fossil fuel energy technologies, ",
                        "and also the weight that these represent in the total amount of energy innovations. ",
                        "You can also use our slider to select a specific year and the maps will adjust accordingly! ",
                        "If you want to know more about a specific country, click on it and a window will pop up with the time trend of the chosen variable for that country.",
                    ],
                    className="main-text",
                ),
                html.P(
                    [
                        "Scroll further down to learn more about our data sources and how we categorize the different variables. ",
                        "Hope you enjoy it!",
                    ],
                    className="main-text",
                ),
                dcc.Dropdown(
                    id="type-selection",
                    options=[
                        {"label": "Renewable Energy Innovation", "value": "renewables"},
                        {"label": "Fossil Fuel Energy Innovation", "value": "ff"},
                        {
                            "label": "Share of Renewable Energy Innovations",
                            "value": "share_renewables",
                        },
                        {
                            "label": "Share of Fossil Fuel Innovations",
                            "value": "share_ff",
                        },
                        
                    ],
                    value="renewables",
                    clearable=False,
                    className="dropdown",
                ),
                dcc.Slider(
                    id="year-slider",
                    min=df["Year"].min(),
                    max=df["Year"].max(),
                    value=df["Year"].max(),  # Set last year as the default value
                    marks={str(year): str(year) for year in df["Year"].unique()},
                    step=None,
                ),
                dcc.Graph(id="world-map"),
                html.Div(
                    id="country-holder"
                ),  # Add hidden Div
                dbc.Modal(
                    [
                        dbc.ModalHeader(id="modal-header"),
                        dbc.ModalBody(dcc.Graph(id="country-trend")),
                    ],
                    id="modal",
                    is_open=False,
                ),
            ]
        ),
        html.Footer(
            [
                html.Div(
                    [
                        html.H4("Data Source", className="footer-heading"),
                        html.P(
                            "PATSTAT is a database that contains bibliographical and legal status patent data from more than 150 countries. It is maintained by the European Patent Office (EPO) and is available to the public for free. PATSTAT contains information on about 150 million patent documents. In particular, information about the inventors, patent classifications and citations are available. The data used in this project was downloaded in March 2021 and covers the period 1978 to 2019.",
                            className="footer-text",
                        ),
                        html.H4("Classification", className="footer-heading"),
                        html.P(
                            [
                                "We allocate a patent to a country based on the nationality of its inventors. ",
                                "If they have different nationalities, we use the nationality that occurs the most. ",
                                "To classify the patents, we use the international patent classification (IPC) ",
                                "and the cooperative patent classification (CPC) systems. ",
                                "For fossil fuel energy innovations, we follow ",
                                "Lanzi et al. (2011)",
                                html.Sup("1"),
                                " who gather the IPC that correspond to fossil fuel energy innovations. ",
                                "For renewable energy innovations, we use the dedicated CPC code.",
                            ],
                            className="footer-text",
                        ),
                        html.P(
                            [
                                html.Small(
                                    html.I(
                                        '1 Lanzi, Elisa, Elena Verdolini, and Ivan Haščič. 2011. "Efficiency-improving fossil fuel technologies for electricity generation: Data selection and trends." Energy Policy, 39(11): 7000–7014.'
                                    )
                                )
                            ],
                            className="footer-text small-font",
                        ),
                    ],
                    className="footer-column",
                ),
                html.Div(
                    [
                        html.H4("Variables", className="footer-heading"),
                        html.P(
                            [
                                html.U("Fossil Fuel Energy Innovation:"),
                                " Number of innovations in fossil fuel energy technologies",
                            ],
                            className="footer-text",
                        ),
                        html.P(
                            [
                                html.U("Renewable Energy Innovation:"),
                                "Number of innovations in renewable energy technologies",
                            ],
                            className="footer-text",
                        ),
                        html.P(
                            [
                                html.U("Share of Fossil Fuel Innovations:"),
                                "Number of innovations in fossil fuel energy technologies relative to the total number of innovations in energy technologies",
                            ],
                            className="footer-text",
                        ),
                        html.P(
                            [
                                html.U("Share of Renewable Energy Innovations:"),
                                "Number of innovations in renewable energy technologies relative to the total number of innovations in energy technologies",
                            ],
                            className="footer-text",
                        ),
                    ],
                    className="footer-column",
                ),
            ]
        ),
        html.Button(id="dummy-button"),
    ]
)


In [604]:
@app.callback(
    Output("dummy-button", "n_clicks"),
    Input("world-map", "clickData"),
)
def update_dummy_button(clickData):
    return 1 if clickData else 0


In [605]:
# World Map Callback


@app.callback(
    Output("world-map", "figure"),
    [Input("type-selection", "value"), Input("year-slider", "value")],
)
def update_world_map(energy_type, selected_year):
    dff = df[df["Year"] == selected_year]

    if energy_type == "ff":
        color_scale = color_scale_ff  # brownish color scale for fossil fuels
    elif energy_type == "renewables":
        color_scale = color_scale_renewables  # custom green color scale for renewables
    elif energy_type == "share_ff":
        color_scale = color_scale_ff
    elif energy_type == "share_renewables":
        color_scale = color_scale_renewables
    else:
        color_scale = (
            px.colors.sequential.Plasma
        )  # default color scale, you can choose any color scale here

    # Define the label based on the energy type
    if energy_type in ["share_ff", "share_renewables"]:
        label = "% of Energy Patents"
    else:
        label = "# of Patents"

    fig = px.choropleth(
    dff,
    locations="Country",
    color=energy_type,
    hover_name="Country_Name",
    projection="natural earth",
    color_continuous_scale=color_scale,
    labels={energy_type: label},  # use the dynamic label here
)

    fig.update_layout(
    clickmode="event+select",  # Enable click events
    font=dict(
        family="Montserrat, sans-serif",
        size=12
    ),
)
    return fig


In [606]:
# This callback handles updating the country-holder when the map is clicked
@app.callback(
    Output("country-holder", "children"),
    [Input("world-map", "clickData")],
)
def update_country_holder(clickData): # Triggered when user clicks on a country. It updates the country-holder with the country code
    if clickData is None:
        return dash.no_update
    return clickData['points'][0]['location']

# A new callback to check if the dropdown or the map was clicked
@app.callback(
    [Output('modal', 'is_open'), Output("country-trend", "figure"), Output("modal-header", "children")],
    [Input("country-holder", "children"), Input("type-selection", "value"), Input('world-map', 'clickData')],
    [State("modal", "is_open")]
)
def check_source(country_code, energy_type, clickData, is_open): # Triggered when country-holder changes, type-selection or world-map is clicked
    ctx = dash.callback_context # check what input triggered the callback
    if not ctx.triggered:
        return dash.no_update # No click yet
    else: 
        trigger_id = ctx.triggered[0]["prop_id"].split(".")[0] 
        if trigger_id == 'world-map': # If the map was clicked, open the modal
            return update_country_trend(country_code, energy_type, is_open)
        elif trigger_id == 'type-selection': # If the dropdown was clicked, either update modal or close it
            if is_open:
                return update_country_trend(country_code, energy_type, is_open)
            else:
                return dash.no_update
        else:
            return dash.no_update

def update_country_trend(country_code, energy_type, is_open): # This function updates the modal
    if country_code is None:
        return is_open, go.Figure(), "Country Trends"
    
    dff = df[df.Country == country_code]
    country_name = dff['Country_Name'].values[0]
    energy_label_dict = {
        'ff': 'Fossil Fuel Energy Innovation', 
        'renewables': 'Renewable Energy Innovation', 
        'share_ff': 'Share of Fossil Fuel Innovations', 
        'share_renewables': 'Share of Renewable Energy Innovations'
    }
    energy_label = energy_label_dict[energy_type]

    # Choose color based on energy type
    if energy_type in ['ff', 'share_ff']:
        line_color = '#D68200'
    else:
        line_color = '#94CFAB'
    
    fig = go.Figure().update_layout(
    font=dict(
        family="Montserrat, sans-serif",
        size=12,
        color="#7f7f7f"
    ),
    margin=dict(t=10, l=50, r=50, b=50),
    # Different y-axis titles for different energy types
    yaxis_title="# of Patents" if energy_type in ['ff', 'renewables'] else "% of Energy Patents",
    xaxis_title="Year",
    plot_bgcolor = '#FFFFFF',
    xaxis = dict(
        gridcolor = '#EAE8E8',
    ),
    yaxis = dict(
        gridcolor = '#EAE8E8',
    )
)
    fig.add_trace(go.Scatter(x=dff["Year"], y=dff[energy_type], mode='lines+markers', name=energy_label, line = dict(color=line_color)))
    
    return (not is_open), fig, "Time Trend of {} for {}".format(energy_label, country_name)


In [607]:
# Run the app

if __name__ == "__main__":
    port = os.getenv("PORT", 8050)
    host = os.getenv("HOST", "localhost")
    app.run_server(debug=True, host=host, port=port)
    print("The app is running at: http://" + host + ":" + str(port))


The app is running at: http://localhost:8050
